# Search FORS2 in IRSA catalogs 

https://astroquery.readthedocs.io/en/latest/ipac/irsa/irsa.html

In [ ]:
import matplotlib.pyplot as plt
from astropy.wcs import WCS

from astroquery.simbad import Simbad
from astroquery.ipac.irsa import Irsa
 

import astropy.coordinates as coordinates
from astropy.coordinates import Angle
import astropy.units as u

import h5py
import pandas as pd
import numpy as np


In [ ]:
from astropy.table.pprint import conf
#conf.max_lines = -1
#conf.max_width = -1

In [ ]:
def GetColumnHfData(hff,list_of_keys,nameval):
    """
    Extract hff atttribute 
    
    parameters
      hff           : descriptor of h5 file
      list_of_keys : list of exposures
      nameval      : name of the attribute
      
    return
           the array of values in the order of 
    """
    

    all_data = []
    for key in list_of_keys:
        group=hff.get(key)
        val=group.attrs[nameval]
        all_data.append(val)
    return all_data


## Fors2 catalog

In [ ]:
input_file_h5  = '../data/FORS2spectra.hdf5'

In [ ]:
hf =  h5py.File(input_file_h5, 'r') 
list_of_keys = list(hf.keys())

In [ ]:
# pick one key    
key_sel =  list_of_keys[0]
# pick one group
group = hf.get(key_sel)  


In [ ]:
#pickup all attribute names
all_subgroup_keys = []
for k in group.attrs.keys():
    all_subgroup_keys.append(k)


In [ ]:
# create info
df_info = pd.DataFrame()
for key in all_subgroup_keys:
    arr=GetColumnHfData(hf, list_of_keys ,key)
    df_info[key] = arr


In [ ]:
df_info.sort_values(by="num", ascending=True,inplace=True)

In [ ]:
df_info_num = df_info["num"].values
key_tags = [ f"SPEC{num}" for num in df_info_num ]


In [ ]:
df_info = df_info[['num', 'ra', 'dec', 'Rmag','redshift']]

In [ ]:
df_info

## IRSA catalog

In [ ]:
Irsa.list_catalogs() 

In [ ]:
Irsa.print_catalogs()

## Target

In [ ]:
target_name_simbad = "BAX 013.5117-28.3994"
target_name_ned = "RX J0054.0-2823"

In [ ]:
result_table = Simbad.query_object(target_name_simbad)

In [ ]:
ra_str = result_table['RA'][0] + ' hours'
dec_str = result_table['DEC'][0] + ' degree'

In [ ]:
coord = coordinates.SkyCoord(ra_str,dec_str)
coord

In [ ]:
result_table = Irsa.query_region(target_name_simbad, catalog="fp_xsc", spatial="Cone",radius=13 * u.arcmin)

In [ ]:
table

In [ ]:
dict_of_catalogs = Irsa.list_catalogs() 

In [ ]:
for cat_key in dict_of_catalogs.keys():
    try:
        result_table = Irsa.query_region(target_name_simbad, catalog=cat_key, spatial="Cone",radius=13 * u.arcmin)
        if len(result_table) >=1:
            print("==============================================================================")
            print("        ",cat_key)
            print(result_table)
    except OSError as err:
        print("OS error:", err)
    except ValueError:
        print("Could not convert data to an integer.")
    except Exception as err:
        print(f"Unexpected {err=}, {type(err)=}")